# 1. high resolution runs
- CESM 1.04
- grid data is stored, dimensions nlat/nlon are natural numbers, but coords U/T_LONG/LAT are in degrees, depths stored in centimeter
- grid data is also stored in binary files

### CTRL: control run
- there are differently sized files:
    - 19GB 024-199 (many files before 100-01 are not available)
    - 21GB 200-226
    - 30GB 227-268

### RCP: rcp CO2 run
- all files are 30 GB

### POP: ocean only run

### interpolated, rect output data
- grid: u/t_lon/lat store position in degrees, depth_t/w_dep store deth in meters
- no further geometric data (such as TAREA or so)

# 2 . low resolution runs
- ocn: 60 vertical levels to 5500 m; nlat: 384, nlon: 320
- atm: different grids!

### LPD: present day control run
`/projects/0/acc/cesm/cesm1_1_2/spinup_B_2000_cam5_f09_g16/OUTPUT`
- CESM 1.12
- 1.02-566.10; several files in the beginning (up to year 153) missing
- atm: lat: 192[-90.,-89.057592,-88.115183,...,90.], lon: 288 [0.,1.25,2.5, ...,356.25,357.5,358.75]

### LPI: pre industrial run
`/projects/0/acc/cesm/cesm1_0_5/b.PI_1pic_f19g16_NESSC_control/OUTPUT`
- CESM 1.05
- ocn: years 1600-2025; before year 1600 data is saved only one year every 50 years
- atm: lat: 96, lev: 26, lon: 144
    - monthly files:
        - `atm/hist/` 2876.02-3024.12
        - `atm/hist/monthly/` 2900-2970.12
    - yearly files:
        - `atm/hist/yearly/` 1550-1600, 1850-1900, 2900-2950, and their averages

In [ ]:
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from grid import generate_lats_lons, shift_field
from paths import file_ex_ocn_ctrl, file_ex_atm_ctrl, file_ex_ocn_rect, file_ex_ocn_rcp
from paths import file_ex_ocn_lpd, file_ex_ocn_lpi, path_CESM
from paths import CESM_filename
from timeseries import IterateOutputCESM
from read_binary import read_binary_2D

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

In [ ]:
file_ex_ocn_ctrl

In [ ]:
ds

In [ ]:
ds.UVEL[0,0,:,:].mean(dim='nlat').plot()

In [ ]:
for y, m, f in IterateOutputCESM(domain='ocn', run='ctrl', tavg='monthly'):
    if os.path.exists(f)==False:
        print(y,m)

In [ ]:
# trying out daily data
for run in ['ctrl', 'rcp']:
    print(f'\n{run}\n')
    if run=='ctrl':  y = 250
    if run=='rcp': y = 2000
    for domain in ['ocn']:
        if domain=='ocn':
            for name in ['SST', 'SSH']:
                print(f'{name}')
                if name=='SST':  d = 31
                if name=='SSH':  d =  1
                fn = CESM_filename(domain=domain, run=run, y=y, m=1, d=d, name=name)
                print(fn)
        else:
            d=1
            fn = CESM_filename(domain=domain, run=run, y=y, m=1, d=d, name=name)
            print(fn)

In [ ]:
for y,m,s in IterateOutputCESM(domain='ocn_rect', run='pop', tavg='monthly'):
    print(y,m,s)

In [ ]:
len(IterateOutputCESM(domain='ocn', run='lpd', tavg='monthly'))/12


In [ ]:
# missing files in LPD run
for y in np.arange(1,567):
#     for m in np.arange(1,13):
    if os.path.exists(CESM_filename(domain='atm', run='lpd', y=y, m=0))==False:
        print(y,m)

In [ ]:
file_ex_ocn_lpd

In [ ]:
ds1 = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)
ds2 = xr.open_dataset(file_ex_ocn_lpi, decode_times=False)

In [ ]:
f, ax = plt.subplots(2,2)
ds1.TLAT .plot(ax=ax[0,0])
ds1.TLONG.plot(ax=ax[1,0])
ds2.TLAT .plot(ax=ax[0,1])
ds2.TLONG.plot(ax=ax[1,1])
plt.tight_layout()

In [ ]:
ds2.dz.plot()

# low resolution files

In [ ]:
ds = xr.open_dataset(file_ex_ocn_rect, decode_times=False)

In [ ]:
ds

In [ ]:
plt.imshow(ds.PD[0,:,:].values)
plt.colorbar()

In [ ]:
ds.TEMP[0,:,:].where(ds.PD[0,:,:]!=np.nan).plot()

In [ ]:
ds.PD[0,1,1].values

In [ ]:
ds.depth_t[0]

In [ ]:
from xr_DataArrays import generate_xr_AREA, generate_xr_DZ

In [ ]:
generate_xr_DZ('ocn_lowres')[0,:,:].plot()

# high resolution files

In [ ]:
# xa0 = xr.open_dataset(file_ex_ohc_hires, decode_times=False)
# xa1 = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
# xa2 = xr.open_dataset(file_ex_ocn_rcp, decode_times=False)
# print(f'ohc file: {len(xa0.variables):3}\n{list(xa0.variables)}')
# print(f'ctrl file: {len(xa1.variables):3}\n{list(xa1.variables)}')
# print(f'rcp file: {len(xa2.variables):3}\n{list(xa2.variables)}')

In [ ]:
# nc_file = f'{path_CESM}/rcp8.5_co2_f05_t12/OUTPUT/ocn/hist/daily/rcp8.5_co2_f05_t12.pop.hm.2021-01-22.nc'
# xdata   = xr.open_dataset(nc_file, decode_times=False)

In [ ]:
xdata = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)

In [ ]:
xdata

In [ ]:
f, ax = plt.subplots(1,3,figsize=(15,5), sharey=True)
ax[0].imshow(xdata.TEMP[0,0,:,:], cmap='viridis')
ax[1].imshow(xdata.coords['TLAT'])
ax[2].imshow(xdata.coords['TLONG'])
plt.tight_layout()

In [ ]:
# does not work with matplotlib 3.0.0, due to '_hold' bug
# use 'CS' conda environment

In [ ]:
# from paths import file_ex_ocn_ctrl

# def all_water_mask():
#     """ creates a tuple of two np.arrays containing all coordinates with water """
#     ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
    
# #     A = np.append(np[0], np.where(xdata.REGION_MASK>0)[0])
# #     B = np.append(np.where(xdata.REGION_MASK<-1)[1], np.where(xdata.REGION_MASK>0)[1])    
# #     assert len(A)==len(B)
    
#     MASK = ds.TAREA[:,:]
#     plt.figure()
#     plt.imshow(MASK)
#     MASK[:,:] = True
#     plt.figure()
#     plt.imshow(MASK)
#     plt.colorbar()
#     temp_MASK = MASK.where(ds.REGION_MASK<-1)
# #     MASK.where(ds.REGION_MASK >0) = True
#     plt.figure()
#     plt.imshow(temp_MASK)
#     plt.colorbar()
        
#     return MASK


In [ ]:
all_water_mask()

In [ ]:
plt.imshow(TEMP)
plt.colorbar()

In [ ]:
print(TEMP[2000,2000].item())
print(TEMP[0,2000].item())

In [ ]:
f = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
xdata.PD[0,0,:,:].where(xdata.REGION_MASK>0).plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), x='TLONG', y='TLAT', add_colorbar=False)
plt.tight_layout()

In [ ]:
def map_geography(ax):
    """adds land, WGKP ploygon, lat/lon labels"""
    xticks = np.linspace(-180,180,7,endpoint=True)
    yticks = np.linspace(-90,90,7)
    ax.add_feature(cartopy.feature.LAND, zorder=1, facecolor='lightgray')
    gl = ax.gridlines(xlocs=xticks, ylocs=yticks)
    gl.n_steps = 500

In [ ]:
# from binary file
grid_file  = f'{path_CESM}/inputdata/ocn/pop/tx0.1v2/grid/horiz_grid_200709.ieeer8'
imt,jmt,km = 3600, 2400, 42
lats, lons, shift = generate_lats_lons(grid_file)
lats,lons  = generate_lats_lons_CESM(grid_file)

In [ ]:
TEMP = shift_field(xdata.TEMP[0,0,:,:].T, shift)
lats_shifted = shift_field(lats, shift)
lons_shifted = shift_field(lons, shift)

In [ ]:
f, ax = plt.subplots(1,3)
ax[0].imshow(lats_shifted)
ax[1].imshow(lons_shifted)
ax[2].imshow(TEMP)

In [ ]:
fig = plt.figure(figsize=(10,10))
# ax  = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
# ax  = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

# ax  = fig.add_subplot(1, 1, 1, projection=ccrs.AzimuthalEquidistant())
ax  = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(central_latitude=-90))
# ax  = fig.add_subplot(1, 1, 1, projection=ccrs.AlbersEqualArea(central_longitude=0.0, central_latitude=0.0, false_easting=0.0, false_northing=0.0, standard_parallels=(20.0, 50.0), globe=None))
ax.set_position([.02,.02,.96,.96])
ax.pcolormesh(xdata.TLONG,
              xdata.TLAT,
              xdata.TEMP[0,0,:,:].where(xdata.REGION_MASK>0),
              cmap='rainbow',
              vmin=-1.8, vmax=27,
              transform=ccrs.PlateCarree() )
map_geography(ax)
# fig.savefig('../../results/first_map', transparent=True, dpi=300)

In [ ]:
fig = plt.figure(figsize=(10,10))
# ax  = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
# ax  = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

ax  = fig.add_subplot(1, 1, 1, projection=ccrs.AzimuthalEquidistant())
# ax  = fig.add_subplot(1, 1, 1, projection=ccrs.AlbersEqualArea(central_longitude=0.0, central_latitude=0.0, false_easting=0.0, false_northing=0.0, standard_parallels=(20.0, 50.0), globe=None))
ax.set_position([.02,.02,.96,.96])
ax.pcolormesh(lons_shifted.T,
              lats_shifted.T,
              TEMP.T,
              cmap='rainbow',
              vmin=-1.8, vmax=27,
              transform=ccrs.PlateCarree() )
map_geography(ax)
# fig.savefig('../../results/first_map', transparent=True, dpi=300)

# Atmosphere

In [ ]:
xdata = xr.open_dataset(file_ex_atm_ctrl, decode_times=False)

In [ ]:
xdata

In [ ]:
plt.imshow(xdata['T'][0,-1,:,:])
plt.colorbar()

In [ ]:
plt.plot(xdata['T'][0,:,300,200])

In [ ]:
xdata['T850'][0,:,:].mean()

In [ ]:
plt.imshow(xdata['T850'][0,:,:])
plt.colorbar()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12,4))
ax[0].imshow(xdata.OCNFRAC[0,:,:], cmap='viridis', origin='lowerleft')
ax[1].imshow(xdata.ICEFRAC[0,:,:], cmap='viridis', origin='lowerleft')
plt.tight_layout()
# plt.colorbar()

In [ ]:
from xr_DataArrays import generate_xr_AREA

In [ ]:
AREA = generate_xr_AREA('atm')

In [ ]:
np.isclose(3.2,3.2,rtol=1e-1)

In [ ]:
plt.plot(AREA[:,0])

In [ ]:
plt.imshow(AREA)
plt.colorbar()

In [ ]:
xdata.lat[0].item()

In [ ]:
plt.plot(xdata.lat[1:].values-xdata.lat[:-1].values)

In [ ]:
xdata.T850

In [ ]:
from xr_integrate import xr_zonal_int, xr_zonal_mean
xa = xdata.T850[0,:,:]
dx = 2
# lat_name = ('lat','lon')
T850_lat_mean = xr_zonal_mean(xa, AREA, dx, 'lat')

# plot the result
plt.plot(T850_lat_mean.coords['lat_bins'], T850_lat_mean)

dx = 5
# lat_name = ('lat','lon')
T850_lat_mean = xr_zonal_mean(xa, AREA, dx, 'lat')

# plot the result
plt.plot(T850_lat_mean.coords['lat_bins'], T850_lat_mean)

In [ ]:
from xr_integrate import xr_vol_int, xr_zonal_int, xr_int_along_axis
from xr_integrate import xr_vol_mean, xr_surf_int, xr_surf_mean

In [ ]:
z = np.array([0,1])
y = np.array([0,1,2])
x = np.array([0,1,2,3])

A = xr.DataArray(data=np.ones((2,3,4)),
                              coords={'z': z, 'y': y, 'x': x},
                              dims=('z', 'y', 'x') )
B = xr.DataArray(data=np.ones((3,4)),
                              coords={'y': y, 'x': x},
                              dims=('y', 'x') )
print(np.all(A[0,:,:]==B[:,:]))

In [ ]:
xr_vol_int(A, B, A, levels=True, zonal=True )

In [ ]:
'lat' in xdata['T'][0,:,:,:]

In [ ]:
A[0,:,:].item

In [ ]:
xr_surf_mean(B, B)